In [ ]:
pip install openai scikit-learn python-dotenv tqdm


In [ ]:
OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
import openai
import json
import os
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Örnek 10 hikaye
hikayeler = [
    "Bir zamanlar küçük bir köyde yaşayan Ali adında bir çocuk vardı...",
    "Derin ormanların arasında kaybolan bir keşif ekibi vardı...",
    "Uzak bir galakside yaşayan robotların dünyası...",
    # ... toplam 10 tane olacak şekilde devam ettir
]

embedding_listesi = []

for hikaye in tqdm(hikayeler):
    response = openai.Embedding.create(
        input=hikaye,
        model="text-embedding-3-small"
    )
    embedding = response["data"][0]["embedding"]
    embedding_listesi.append({
        "text": hikaye,
        "embedding": embedding
    })

# JSON olarak kaydet
with open("hikayeler_embedding.json", "w", encoding="utf-8") as f:
    json.dump(embedding_listesi, f, ensure_ascii=False, indent=2)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Kullanıcıdan soru al
soru = input("Sorunuzu girin: ")

# Soru embedding
soru_embedding = openai.Embedding.create(
    input=soru,
    model="text-embedding-3-small"
)["data"][0]["embedding"]

# Daha önce kaydedilen hikayeleri yükle
with open("hikayeler_embedding.json", "r", encoding="utf-8") as f:
    hikaye_verisi = json.load(f)

# Cosine similarity hesapla
benzerlikler = []
for item in hikaye_verisi:
    cos_sim = cosine_similarity(
        [soru_embedding],
        [item["embedding"]]
    )[0][0]
    benzerlikler.append(cos_sim)

# En yüksek benzerliği bul
en_yakin_index = np.argmax(benzerlikler)
en_yakin_hikaye = hikaye_verisi[en_yakin_index]["text"]

print(f"\nEn yakın hikaye:\n{en_yakin_hikaye}")
print(f"\nBu cevap RAG verisine dayanmaktadır. Cosine Benzerlik: {benzerlikler[en_yakin_index]:.4f}")


In [ ]:
cevap = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "Sen bir yardımcı asistansın. Verilen hikâyeyi kullanarak kullanıcının sorusunu yanıtla. Eğer cevap hikâyeden alınamıyorsa 'Bu hikâyede doğrudan bir yanıt bulunmamaktadır.' de."},
        {"role": "user", "content": f"Hikaye:\n{en_yakin_hikaye}\n\nSoru:\n{soru}"}
    ]
)

print("\n🔍 Cevap:")
print(cevap["choices"][0]["message"]["content"])
